In [1]:
Pkg.add("Fontconfig");
using Distributions;
mu = [10 20 30 40]';
Lambda =
[1.0 0 0
 0 1.0 0
 0 0 1.0
 0.5 0.5 0];
Psi = diagm([0.1, 0.2, 0.3, 0.4]);
d1 = MvNormal([0,0,0],ones(3));
X = zeros(50,4);
for i=1:50
    f = rand(d1,1);
    d2 = MvNormal(vec(mu+ Lambda*f),Psi);
    x = rand(d2,1);
    X[i,:] = x';
end

INFO: Pkg operations are not possible on JuliaBox. Please use the "Packages" menu at the top of the main screen.


In [2]:
X

50×4 Array{Float64,2}:
 10.2135   18.2068  30.8905  38.9934
 10.8788   19.6114  29.1003  40.4984
 10.6902   20.8779  30.2459  40.998 
 10.4516   21.6165  31.685   40.1984
  9.2839   20.8496  31.1147  39.6304
 11.9288   21.8875  28.7381  41.9217
  9.20865  20.5435  29.5727  39.0525
  9.28215  20.618   29.0142  40.6504
 10.4521   20.5248  31.3456  40.2524
  9.20026  21.1043  30.7307  40.1446
 11.0201   21.0918  31.784   40.7203
  8.9281   19.3562  29.327   39.3728
  9.87453  17.7911  29.1338  38.8467
  ⋮                                 
 10.5772   22.2035  30.1235  40.7876
  9.98442  19.3122  29.7339  40.1105
  9.93499  19.6806  29.5686  39.8495
  9.93308  17.6796  29.831   39.0399
 10.6361   18.7816  31.0701  39.8821
 11.8275   18.8411  30.2213  40.1822
 10.4591   18.8053  28.9624  40.3793
 11.6365   19.3121  29.98    40.293 
 11.0035   18.4654  31.3973  40.0051
  8.94059  21.115   28.7417  39.8501
 11.364    19.8672  30.0691  40.5178
 10.2412   18.2768  28.8037  38.9638

E step

In [3]:
function E_Step(X,mu,Lambda,Psi,k)
    mu_f_by_x = (X - repmat(mu',size(X,1),1))*(Lambda'*inv(Lambda*Lambda' + Psi))';
    Sig_f_by_x = eye(k) - Lambda'*inv(Lambda*Lambda' + Psi)*Lambda;
    return mu_f_by_x,Sig_f_by_x;
end

E_Step (generic function with 1 method)

In [4]:
function M_Step(X,mu_f_by_x,Sig_f_by_x,k)
    nrows, ncols = size(X);
    #Computing mu
    mu = mean(X,1)';
    #Computing Lambda
    Lambda_term1 = zeros(ncols,k);
    Lambda_term2 = zeros(k,k);
    for i=1:nrows
        Lambda_term1 = Lambda_term1 + ((X[i,:] - mu)*mu_f_by_x[i,:]');
        Lambda_term2 = Lambda_term2 + inv((mu_f_by_x[i,:]*mu_f_by_x[i,:]')+Sig_f_by_x);
    end
    Lambda = Lambda_term1*(Lambda_term2);
    #Computing Psi
    Phi = zeros(ncols,ncols);
    for i=1:nrows
    Phi = Phi + (X[i,:]*X[i,:]' - X[i,:]*mu_f_by_x[i,:]'*Lambda' - Lambda*mu_f_by_x[i,:]*X[i,:]' + Lambda*(mu_f_by_x[i,:]*mu_f_by_x[i,:]'+Sig_f_by_x)*Lambda')
    end
    Psi = diagm(diag(Phi./nrows));
    return mu, Lambda, Psi
end

function compute_llh(X,mu,Lambda,Psi)
    llh = 0;
    for i=1:size(X,1)
    llh = llh + log(pdf(MvNormal(vec(mu),(Lambda*Lambda')+Psi),X[i,:]));
    end
    return llh;
end

compute_llh (generic function with 1 method)

In [8]:
function fa_em(X,k)
    max_Iter = 100;
    eps = 0.0001;
    llh = -Inf*ones(max_Iter+1);
    mu = mean(X,1)';
    Lambda = rand(size(X,2),k);
    Psi = diagm(rand(size(X,2)));
    print(mu,"\n",Lambda,"\n",Psi,"\n");
    llh[1] = compute_llh(X,mu,Lambda,Psi);
    print(llh[1],"\n")
    for i=1:max_Iter
        print(i,"\n");
        mu_f_by_x,Sig_f_by_x = E_Step(X,mu,Lambda,Psi,k);
        mu_new, Lambda_new, Psi_new = M_Step(X,mu_f_by_x,Sig_f_by_x,k);
        print(mu_new,"\n",Lambda_new,"\n",Psi_new,"\n");
        llh[i+1] = compute_llh(X,mu_new,Lambda_new,Psi_new);
        print(llh[i+1],"\n");
        if(sum(abs.(mu_new-mu))<eps && sum(abs.(Lambda_new-Lambda))<eps && sum(abs.(Psi_new-Psi))<eps)
            break;
        end
        mu = mu_new;
        Lambda = Lambda_new;
        Psi = Psi_new;
    end
    mu_f_by_x,Sig_f_by_x = E_Step(X,mu,Lambda,Psi,k);
    return mu, Lambda, Psi, mu_f_by_x, Sig_f_by_x, llh;
end

#Calling the EM approach for dataset X and 2 factors
mu, Lambda, Psi, mu_f_by_x, Sig_f_by_x, llh = fa_em(X,3)

using Gadfly, Cairo
#plot the log-likelihood
plot(x=collect(1:1:101), y=llh,Geom.line)


[10.1608; 19.9515; 30.1157; 40.0723]
[0.533554 0.599178 0.709342; 0.820012 0.415963 0.10249; 0.60397 0.471141 0.550533; 0.596754 0.349353 0.3636]
[0.871595 0.0 0.0 0.0; 0.0 0.970362 0.0 0.0; 0.0 0.0 0.604083 0.0; 0.0 0.0 0.0 0.782682]
-300.4461898026349
1

LoadError: [91mBase.LinAlg.PosDefException(3)[39m


[10.1608; 19.9515; 30.1157; 40.0723]
[1489.84 1520.42 1876.2; 2834.27 1300.55 266.735; 1550.56 1291.87 1628.31; 2241.68 1483.56 1252.46]
[4.26491e6 0.0 0.0 0.0; 0.0 6.57316e6 0.0 0.0; 0.0 0.0 3.50686e6 0.0; 0.0 0.0 0.0 4.7137e6]
-1779.7103613040558
2
[10.1608; 19.9515; 30.1157; 40.0723]
[1.663 1.44174 1.61132; 2.4252 1.26803 0.589882; 1.26394 1.06058 1.32994; 2.2174 1.5074 1.2633]
[105.31 0.0 0.0 0.0; 0.0 401.077 0.0 0.0; 0.0 0.0 908.703 0.0; 0.0 0.0 0.0 1607.87]
-807.6989314890703
3
[10.1608; 19.9515; 30.1157; 40.0723]
[42.3708 36.4614 40.5119; 24.682 12.9792 6.16795; 7.43364 6.17669 7.17628; 35.7038 27.1222 26.4788]
[4451.07 0.0 0.0 0.0; 0.0 1151.92 0.0 0.0; 0.0 0.0 1040.22 0.0; 0.0 0.0 0.0 4077.96]
-985.0841919828263
4
[10.1608; 19.9515; 30.1157; 40.0723]
[37.5434 31.3627 33.9875; 96.2745 53.8448 30.8742; 22.4191 18.1574 20.4388; 66.5357 44.0792 35.7229]
[1155.48 0.0 0.0 0.0; 0.0 4656.91 0.0 0.0; 0.0 0.0 1278.98 0.0; 0.0 0.0 0.0 3831.49]
-1018.2936733510152
5
[10.1608; 19.9515; 30.

LoadError: [91mUndefVarError: llh not defined[39m